# CONTENEDORES
## Que es un contenedor ?

- Es una agrupación de procesos.

- Es una entidad lógica, no tiene el limite estricto de las máquinas virtuales, emulación del sistema operativo simulado por otra más abajo.

- Ejecuta sus procesos de forma nativa.

- Los procesos que se ejecutan adentro de los contenedores ven su universo como el contenedor lo define, no pueden ver mas allá del contenedor, a pesar de estar corriendo en una maquina más grande.

- No tienen forma de consumir más recursos que los que se les permite. Si esta restringido en memoria ram por ejemplo, es la única que pueden usar.

- A fines prácticos los podemos imaginar cómo maquinas virtuales, pero NO lo son. Máquinas virtuales livianas.

- Docker corre de forma nativa solo en Linux.

- Sector del disco: Cuando un contenedor es ejecutado, el daemon de docker le dice, a partir de acá para arriba este disco es tuyo, pero no puedes subir mas arriba.

- Docker hace que los procesos adentro de un contenedor este aislados del resto del sistema, no le permite ver más allá.

- Cada contenedor tiene un ID único, también tiene un nombre.
<img src="https://devopedia.org/images/article/101/8323.1565281088.png" width="650px">

## Ciclo de vida de un contenedor
Cada vez que un contendor se ejecuta, en realidad lo que ejecuta es un proceso del sistema operativo. Este proceso se le conoce como Main process.

### Main process
Determina la vida del contenedor, un contendor corre siempre y cuando su proceso principal este corriendo.
El contenedor se apaga cuando su proceso principal (main process) hace "exit",
para correr un contenedor que no se apaga automaticamente, solo utilizando el modo iterativo. Se hace correr otro proceso no el bin/bash/ que no haga exit

### Sub process
Un contenedor puede tener o lanzar procesos alternos al main process, si estos fallan el contenedor va a seguir encedido a menos que falle el main.

Ejemplos manejados en el video
- Batch como Main process
- Agujero negro (/dev/null) como Main process

```
docker run --name alwaysup -d ubuntu tail -f /dev/null
-d --detach: 
tail -f /dev/null: espesifico el comando que corre
```


## Interfaz de red
Cada contenedor tiene su interfaz de red, para conectar el puerto del contenero al exterior, se usa el comando publis (-p)
```
docker run -p <public port>:<container port> container
```

## Importacion y exportacion de archivos de un contenedor
<img src="https://i1.wp.com/cdn-images-1.medium.com/max/800/1*bo6IOrBjaHbtkPgTKT08NA.png?w=1170&ssl=1" width="500px">

`Host:` Donde Docker esta instalado.<br>
`Bind Mount:` Guarda los archivos en la maquina local persistiendo y visualizando estos datos (No seguro).<br>
`Volume:` Guarda los archivos en el area de Docker donde Docker los administra (Seguro).<br>
`TMPFS Mount:` Guarda los archivos temporalmente y persiste los datos en la memoria del contenedor, cuando muera sus datos mueren con el contenedor.<br>


# Datos en Docker
## Volumenes 
 Los volúmenes son una forma de persistir y compartir datos entre contenedores y el host. Los volúmenes de Docker son directorios o archivos en el host que se montan en el contenedor y se utilizan para almacenar y compartir datos de forma persistente.
 Al utilizar volúmenes, los datos almacenados en ellos son independientes del ciclo de vida del contenedor, lo que significa que los datos persisten incluso después de que el contenedor se detiene o se borra. Esto es útil para aplicaciones que necesitan almacenar datos persistentes, como bases de datos, archivos de configuración, archivos de registro, etc



# Imagenes
Una imágen contiene distintas capas de datos (distribución, diferente software, librerías y personalización).
Un contenedor es un objecto una instancia, y la imagen es la clase.
Podemos llegar a la conslusión, que una imágen se conforma de distintas capas de personalización, en base a una capa inicial (base image), la dicha capa, es el más puro estado del SO.

<img src="https://static.packt-cdn.com/products/9781788992329/graphics/0ee3d4cf-2133-4143-a7c4-690274483841.png" width="500px">

Las imagenes podemos sacarlas de <a href="https://hub.docker.com/">dockerhub</a>

## Estructura basica de un Dockerfile
`1. FROM:` La primera línea del Dockerfile especifica la imagen base desde la que se construirá la nueva imagen. Por ejemplo, FROM ubuntu:latest utilizará la última versión de la imagen base Ubuntu.

`2. LABEL:` Esta sección permite agregar metadatos a la imagen, como el autor, la versión y una descripción.

`3. RUN:` Aquí se pueden incluir comandos que se ejecutarán durante la construcción de la imagen para instalar paquetes, configurar el sistema y realizar otras tareas.

`4. COPY o ADD:` Esta sección se utiliza para copiar archivos desde el sistema local al contenedor Docker.

`5. WORKDIR:` Esta línea establece el directorio de trabajo dentro del contenedor donde se ejecutarán los comandos.

`6. EXPOSE:` Aquí se especifican los puertos que deben estar expuestos para permitir que otros contenedores o máquinas se comuniquen con el contenedor.

`7. CMD o ENTRYPOINT:` Esta sección especifica el comando que se ejecutará al iniciar el contenedor. Si se utiliza CMD, el comando se puede reemplazar al ejecutar el contenedor con un comando diferente. Si se utiliza ENTRYPOINT, el comando se ejecuta siempre, pero se pueden pasar argumentos adicionales.

```
FROM ubuntu:latest
LABEL maintainer="Tu nombre <tu.email@example.com>"
RUN apt-get update && apt-get install -y python3
COPY . /app
WORKDIR /app
EXPOSE 80
CMD ["python3", "app.py"]
```

## Aprovechando el caché de capas para estructurar correctamente tus imágenes
Cada vez que se hace un cambio en Dockerfile o en los archivos de la aplicaciones, al volver a generar el proyecto se instala todo nuevamente

## Docker compose
permite definir y ejecutar aplicaciones Docker multi-contenedor. Es una herramienta muy útil para orquestar aplicaciones complejas que se componen de varios servicios y contenedores. 
Podemos definir los servicios, la configuración de red y las variables de entorno necesarias para cada contenedor de forma declarativa en un archivo YAML.
Estructura básica de un archivo docker-compose.yml, que define dos servicios: una aplicación Node.js y una base de datos MongoDB:
```
version: '3'
services:
  app:
    build: .
    ports:
      - "3000:3000"
    environment:
      - MONGO_URL=mongodb://dbdata:27017/custom
    depends_on:
      - db
  db:
    image: mongo:latest
    volumes:
      - data:/data/db
volumes:
  data:
```
Este archivo define dos servicios: "app" y "db".
- El servicio "app" se construirá a partir de un Dockerfile en el directorio actual (indicado por el .), expondrá el puerto 3000 y establecerá la variable de entorno MONGO_URL en el valor mongodb://dbdata:27017/custom. El servicio depende del servicio "db".

- El servicio "db" se ejecutará utilizando la imagen "mongo:latest" y se vinculará el volumen "data" al directorio "/data/db" en el contenedor.

### Network
LA red que se crea va a contener a los servicios y su nombre al ejecutar el comando `docker network ls` sera:

<Folder_project_name>_default


### Volumenes
Para montar los volumenes se debe usar la sentencia volumenes
```
volumes:
      - .:/usr/src/
      - /user/src/node_module
```
- Si esta con los dos puntos `:`, implica que va a montar los datos
- Si no tiene los dos puntos `:`, va a ignorar dicho archivo